In [473]:
import numpy as np
import pandas as pd


In [474]:
df = pd.read_csv('checkins.dat',delimiter='|',header = 0)

/Users/alexeyzuev/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [475]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,,,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.8951118,-77.0363658,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,,,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.41052,2012-04-21 17:43:43


In [476]:
df.columns = df.columns.str.replace(' ', '')

In [477]:
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

In [478]:
df = df.dropna()

In [479]:
df.shape

(396634, 6)

In [480]:
df = df[:100000]

In [457]:
from sklearn.cluster import MeanShift, estimate_bandwidth

ms = MeanShift(bandwidth=0.1) 

In [458]:
labels = ms.fit_predict(df[['latitude','longitude']])

In [459]:
ms.cluster_centers_.shape

(3231, 2)

In [460]:
df['cluster'] = labels

In [461]:
cluster_centers = ms.cluster_centers_[df['cluster'].value_counts() >=15]

In [462]:
# Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов: 
# Я сохранил адреса в файле
with open('carnival_ad.txt', 'r') as f:
    file_data = f.readlines()

In [463]:
import re
office_centers = [re.findall("\d+\.\d+", line) for line in file_data]
office_centers = [[float(i) for i in line] for line in centers]

In [464]:
import math
distances_from_offices = []
best_office_distance = 1e6
best_cluste_location= 0

for cluster_center in cluster_centers:
    min_dist = 1e6
    closest_office_coordinates = 0
    
    for office_center in office_centers:
        dist = math.hypot(cluster_center[0] - office_center[0], cluster_center[1] - office_center[1])
        if dist < min_dist:
            min_dist = dist
            closest_office_coordinates = office_center
    # можно все расстояния сохранять в массиве    
    distances_from_offices.append([cluster_center,min_dist,closest_office_coordinates])
    
    # а можно просто искать сразу самый ближний к офису центр кластера
    if min_dist < best_office_distance:
        best_office_distance = min_dist
        best_cluster_location = cluster_center
                    


In [465]:
# а можно все записать в пандас дф.
distances_from_offices = pd.DataFrame(distances_from_offices,columns=['cluster_center','distance','closest_office'])   
distances_from_offices.sort_values(by=['distance'])[:1]

,cluster_center,distance,closest_office
370,"[52.37296399032261, 4.892317222580647]",0.009353,"[52.378894, 4.885084]"


In [466]:
best_cluster_location

array([52.37296399,  4.89231722])

In [467]:
best_office_distance

0.009353316185992226